In [ ]:
import time
import os
import threading
import pyaudio
import socket

# i act like a server who sends data
# i expose my port for other units to connect
# i won't send right data now
def initConnect(ip='localhost', port=5000):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    address = (ip, port)
    s.bind(address)
    s.listen(5)
    print('Ready to transmit. Waiting for connecting...')
    conn, addr = s.accept()
    print('Get a connection from: ', addr)
    return conn,addr, s

# i act like a client who receives data
# i tend to connect server's port
# some other function will receive data via my built connection
def initConnected(ip='localhost', port=6000):
    r = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    address = (ip, port)
    r.connect(address)
    print("Ready to receive.")
    return r 

# i act a sampling device
# i record voice
# i won't record right now
def initSamplingDevice(iformat=pyaudio.paInt16, ichannel=1, isampleRate=16000, ichunk=1280):
    handler = pyaudio.PyAudio()
    stream = handler.open(format = iformat, 
                          channels = ichannel, 
                          rate = isampleRate, 
                          input =True, 
                          frames_per_buffer = ichunk)
    print("Reocrd device ready.")
    return handler, stream

# i act a playing device
# i play stream
# i won't play right now
def initPlayingDevice(iformat=pyaudio.paInt16, ichannel=1, isampleRate=16000, ichunk=1280):
    handler = pyaudio.PyAudio()
    stream = handler.open(format = iformat, 
                          channels = ichannel, 
                          rate = isampleRate, 
                          output = True,
                          #output_device_index = 0,
                          frames_per_buffer = ichunk)
    print("Playing device ready.")
    return handler, stream

def transmitSingleChunk(conn, data):
    if data!='':
        conn.send(data)
        
def receiveSingleChunk(receiver, ichunk):
    data = receiver.recv(ichunk)
    return data

def transmitStringChunk(conn, receiver, data):
    #how long is data?
    lenData = len(bytes(data, 'utf-8'))
    #need to tell receiver how long is this coming chunk
    header = int.to_bytes(lenData, 4 , byteorder = 'big', signed = False)
    #print(type(header))
    conn.sendall(header)
    #need to wait until i heard response where response equals to what i sent before
    while(1):
        tempData = receiver.recv(4)
        response = int.from_bytes(tempData, byteorder = 'big', signed = False)
        if response == lenData:
            break;
    #send full message 
    #time.sleep(0.1)
    conn.sendall(bytes(data, 'utf-8'))
    #time.sleep(0.1)
    
def receiveStringChunk(conn, receiver):
    tempData = receiver.recv(4)
    response = int.from_bytes(tempData, byteorder = 'big', signed = False)
    conn.sendall(tempData)
    data = receiver.recv(response)
    print(str(data,'utf-8'))

def keep_sending(conn, receiver):
    i = 0
    while(1):
        i = i+1
        time.sleep(2)
        data = "从Entity1传过来的第#" + str(i)+"数据"
        transmitStringChunk(conn, receiver, data)

def keep_receiving(conn, receiver):  
    while(1):
        receiveStringChunk(conn, receiver)
        
        
def keep_sending_audio(conn):      
    _, stream = initSamplingDevice();
    while(1):
        data = stream.read(1280)
        #print(data)
        conn.send(data)
        
def keep_receiving_audio(receiver):
    _, stream = initPlayingDevice();
    while(1):
        data = receiver.recv(1280)
        stream.write(data)
        print(data)
        
        
        
#E1 send message E2 receive
print("connecting message service...")
connA2B, _, sA2B = initConnect(ip='169.254.154.25', port=8000)
time.sleep(1)
rA2B = initConnected(ip='169.254.120.218', port=8001)
connB2A, _, sB2A = initConnect(ip='169.254.154.25', port=8010)
time.sleep(2)
rB2A = initConnected(ip='169.254.120.218', port=8011)
print("connecting audio service...")
AconnA2B, _, AsA2B = initConnect(ip='169.254.154.25', port=9000)
time.sleep(1)
ArA2B = initConnected(ip='169.254.120.218', port=9001)
AconnB2A, _, AsB2A = initConnect(ip='169.254.154.25', port=9010)
time.sleep(2)
ArB2A = initConnected(ip='169.254.120.218', port=9011)
print("Done")

t1 = threading.Thread(target = keep_sending, kwargs = {"conn":connA2B, "receiver":rA2B})
t1.start()
time.sleep(3)
t2 = threading.Thread(target = keep_receiving, kwargs = {"conn":connB2A, "receiver":rB2A} )
t2.start()

time.sleep(1)
#ime.sleep(0.5)
At1 = threading.Thread(target = keep_sending_audio, kwargs = {"conn":AconnA2B})
At1.start()

#At2 = threading.Thread(target = keep_receiving_audio, kwargs = {"receiver":ArB2A})
#At2.start()
#time.sleep(2)



